IMPORTS

In [12]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import files

Loading Data

In [3]:
# Step 1: Load the actual column headers (from second row, index 1)
header_row = pd.read_csv('Data/MILCO_SALES_2024_2025.csv', skiprows=0, nrows=1)

# Clean header strings: remove weird characters like non-breaking space or en-dash
cleaned_columns = (
    header_row.columns
    .astype(str)
    .str.strip()
    .str.replace('\u00a0', '', regex=False)  # remove non-breaking space
    .str.replace('\u2013', '-', regex=False) # en-dash
    .str.replace('–', '-', regex=False)      # literal en-dash
)

# Step 2: Load the actual data (skip 2 rows)
df = pd.read_csv('Data/MILCO_SALES_2024_2025.csv', skiprows=[0, 1], header=None)
df.columns = cleaned_columns

# Rename first two columns
df = df.rename(columns={df.columns[0]: 'Product', df.columns[1]: 'Size'})

# Drop footer blank rows
df = df.dropna(subset=['Product']).reset_index(drop=True)

# Get all columns except 'Product' and 'Size'
sales_cols = [col for col in df.columns if col not in ['Product', 'Size']]

# Convert sales values to numeric (remove commas)
df[sales_cols] = df[sales_cols].replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Melt to long format
df_long = pd.melt(
    df,
    id_vars=['Product', 'Size'],
    value_vars=sales_cols,
    var_name='Date',
    value_name='Sales'
)

# Clean the Date strings before parsing
df_long['Date'] = (
    df_long['Date']
    .astype(str)
    .str.strip()
    .str.replace('\u00a0', '', regex=False)
    .str.replace('\u2013', '-', regex=False)
    .str.replace('–', '-', regex=False)
)

# Convert to datetime
df_long['Date'] = pd.to_datetime(df_long['Date'], format='%b-%y', errors='coerce')

# Final tidy
df_long = df_long.sort_values(['Product', 'Date']).reset_index(drop=True)

# Preview
print(df_long.head())
print(f"{df_long['Date'].isna().sum()} dates failed to parse.")


             Product    Size       Date     Sales
0  Butter (Salted )   200 gm 2024-01-01   48200.0
1  Butter (Salted )   200 gm 2024-02-01   18200.0
2  Butter (Salted )   200 gm 2024-03-01   45000.0
3  Butter (Salted )   200 gm 2024-04-01  103800.0
4  Butter (Salted )   200 gm 2024-05-01   71600.0
0 dates failed to parse.


Handling N/A values

In [4]:
na_counts = df_long.isna().sum()
print("Missing values per column:\n", na_counts)

Missing values per column:
 Product    0
Size       0
Date       0
Sales      6
dtype: int64


In [5]:
missing_rows = df_long[df_long.isna().any(axis=1)]
print(missing_rows.head())

                    Product    Size       Date  Sales
18       Butter (Unsalted)   200 gm 2024-02-01    NaN
19       Butter (Unsalted)   200 gm 2024-03-01    NaN
392  M/ Powder Highland Ela  400 gm 2024-02-01    NaN
407  M/ Powder Highland Ela  400 gm 2025-05-01    NaN
419      M/P- Non Fat-       400 gm 2024-12-01    NaN


In [7]:
df_long = df_long.fillna(0)
print("Any missing values left?", df_long.isna().any().any())

Any missing values left? False


CREATING LAG FEATURE AND ROLLING AVERAGE

In [8]:
# Sort before creating lag/rolling features
df_long = df_long.sort_values(['Product', 'Size', 'Date'])

# Create lag features
for lag in [1, 2, 3, 12]:
    df_long[f'lag_{lag}'] = (
        df_long
        .groupby(['Product', 'Size'])['Sales']
        .shift(lag)
        .fillna(0)
    )

# Create rolling averages
df_long['roll_3'] = (
    df_long
    .groupby(['Product', 'Size'])['Sales']
    .shift(1)                         # exclude current month
    .rolling(window=3, min_periods=1)
    .mean()
    .fillna(0)
)

df_long['roll_6'] = (
    df_long
    .groupby(['Product', 'Size'])['Sales']
    .shift(1)
    .rolling(window=6, min_periods=1)
    .mean()
    .fillna(0)
)


In [9]:
df_long.head()

,Product,Size,Date,Sales,lag_1,lag_2,lag_3,lag_12,roll_3,roll_6
0,Butter (Salted ),200 gm,2024-01-01,48200.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Butter (Salted ),200 gm,2024-02-01,18200.0,48200.0,0.0,0.0,0.0,48200.000000,48200.000000
2,Butter (Salted ),200 gm,2024-03-01,45000.0,18200.0,48200.0,0.0,0.0,33200.000000,33200.000000
3,Butter (Salted ),200 gm,2024-04-01,103800.0,45000.0,18200.0,48200.0,0.0,37133.333333,37133.333333
4,Butter (Salted ),200 gm,2024-05-01,71600.0,103800.0,45000.0,18200.0,0.0,55666.666667,53800.000000
